In [18]:
import scanpy as sc
import pandas as pd
import re

In [19]:
### day 30
file = '/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.h5'
adatafull_d30 = sc.read(file)
adatafull_d30.shape

(250923, 32738)

In [20]:
### day 52
file = '/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_17_D52/pool1_17_D52.scanpy.h5'
adatafull_d52 = sc.read(file)

In [21]:
adatafull_d52.shape

(523094, 32738)

In [22]:
# adatafull_d52.obs

In [23]:
##### cells from top pseudocells for eQTL XX only
flip_folder = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/"

In [24]:
import glob
myfiles = glob.glob(flip_folder+"*map.csv")

In [25]:
myfiles

['/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/C19orf10-19_4664038_T_C_bottom20quantile_pseudocell_cell_map.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/SLC18A2-10_119020918_A_G_top20quantile_pseudocell_cell_map.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/HSD17B4-5_118804583_C_A_bottom20quantile_pseudocell_cell_map.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/RBM26-AS1-13_79980340_C_T_top20quantile_pseudocell_cell_map.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/HSPA4-5_132428503_T_C_top20quantile_pseudocell_cell_map.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_s

In [ ]:
############## option 1 (simple):
### aggregate at donor level, irrespective of condition (d30, d52, d52ROT)
### only 1/n as covariance

In [48]:
for file in myfiles:
    df_cell = pd.read_csv(file, index_col=0)
    selected_cells = list(df_cell['cell'])
    cells_d30 = sorted(set(list(adatafull_d30.obs.index.values)).intersection(selected_cells))
    cells_d52 = sorted(set(list(adatafull_d52.obs.index.values)).intersection(selected_cells))
    adata_d30 = adatafull_d30[cells_d30].copy()
    adata_d52 = adatafull_d52[cells_d52].copy()
    adata = adata_d30.concatenate(adata_d52, batch_key='timepoint')
    # make pseudobulk matrix
    genes = adata.var_names
    donors = adata.obs['donor_id'].astype("category").cat.categories
    res = pd.DataFrame(columns=genes, index=donors)
    # loop over donors
    for donor in donors: 
        res.loc[donor] = adata[adata.obs['donor_id'].isin([donor]),:].X.mean(0)
    res['n_cells'] = 0
    for donor in donors: 
        res.n_cells[donor] = adata[adata.obs['donor_id'].isin([donor]),:].shape[0]
    file_new = re.sub("pseudocell_cell_map", "mean_donor_level", file)
    res.to_csv(file_new)

/nfs/software/stegle/users/acuomo/conda-envs/spatialde/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
